In [1]:
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras import backend as K
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from data.preprocess import *
from evaluation.evaluate import *
from evaluation.benchmark import *
data = preprocess(path='../data/raw/CD_PD.mat')

In [2]:
K.clear_session()

## Predict label PD diagnosis

In [3]:
def get_age_gender_data(data):
    #get shuffled data
    data_train, data_test = split_by_patient(data)
    
    #label PD
    y_test = data_test['diagnosis'].astype(np.float32)
    y_train = data_train['diagnosis'].astype(np.float32)
    
    age_train = data_train['age'].astype(np.float32)
    age_train = age_train.reshape(1, *age_train.shape).T
    gender_train = data_train['gender'].astype(np.float32)
    gender_train = gender_train.reshape(1, *gender_train.shape).T

    age_test = data_test['age'].astype(np.float32)
    age_test = age_test.reshape(1, *age_test.shape).T
    gender_test = data_test['gender'].astype(np.float32)
    gender_test = gender_test.reshape(1, *gender_test.shape).T
    
    #x
    x_train = np.concatenate((age_train,gender_train),axis=1)
    x_test = np.concatenate((age_test, gender_test), axis=1)
    
    return x_test, x_train, y_test, y_train

In [4]:
x_test, x_train, y_test, y_train = get_age_gender_data(data)
x_train.shape, x_test.shape, y_test.shape, y_train.shape

((156, 2), (40, 2), (40,), (156,))

# Dense1 (age + gender)

In [5]:
def get_nn_model():
    #optimizer
    adam = Adam(lr=1e-4)
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(2,)))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss=MSE, optimizer=adam)
    model.build()
    return model
_ = get_nn_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               300       
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1

# Performance Evaluation

We will run the model training & inference k times and evaluate the average performance 

In [6]:
models = run_benchmark(get_age_gender_data, get_nn_model, data, k=30, model_name='Dense1 (age+gender)')

# Dense 2 (age + gender + Ed + TMSE)